In [17]:
import os
#spark imports
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
import csv
from pyspark.sql.types import *
from pyspark.sql.functions import format_number, when
import pandas as pd
import matplotlib.pyplot as plt
import os
#spark imports
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
import csv
from pyspark.sql.types import *
from pyspark.sql.functions import format_number, when
import pyspark.sql.functions as F
import pandas as pd
import matplotlib.pyplot as plt


In [18]:
#change RAM allocation, current = 8GB
#change Core allocation, current = 3 Cores
spark = SparkSession.builder.appName("Project - Chicago crime")\
.config("spark.some.config.option", "some-value")\
.config("spark.driver.memory", "8g")\
.config("spark.driver.cores", '3')\
.getOrCreate()

In [19]:
crimes_schema = StructType([StructField("ID", StringType(), True),
                            StructField("Case Number", StringType(), True),
                            StructField("Date", StringType(), True ),
                            StructField("Block", StringType(), True),
                            StructField("IUCR", StringType(), True),
                            StructField("Primary Type", StringType(), True  ),
                            StructField("Description", StringType(), True ),
                            StructField("Location Description", StringType(), True ),
                            StructField("Arrest", BooleanType(), True),
                            StructField("Domestic", BooleanType(), True),
                            StructField("Beat", StringType(), True),
                            StructField("District", StringType(), True),
                            StructField("Ward", StringType(), True),
                            StructField("Community Area", StringType(), True),
                            StructField("FBI Code", StringType(), True ),
                            StructField("X Coordinate", DoubleType(), True),
                            StructField("Y Coordinate", DoubleType(), True ),
                            StructField("Year", IntegerType(), True),
                            StructField("Updated On", DateType(), True ),
                            StructField("Latitude", DoubleType(), True),
                            StructField("Longitude", DoubleType(), True),
                            StructField("Location", StringType(), True )
                            ])

In [20]:
#set your local dataset path
dataSetPath = '/Applications/devActivities/BigDataSpark/Project/Crimes_-_2001_to_present.csv'

In [21]:
dataset = spark.read.option("header", "True")\
        .option("schema", crimes_schema)\
        .csv(dataSetPath)

### Have a look at the data schema:

In [22]:
dataset.limit(19).toPandas().head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,11034701,JA366925,01/01/2001 11:00:00 AM,016XX E 86TH PL,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,false,false,...,8,45,11,None,None,2001,08/05/2017 03:50:08 PM,None,None,None
1,11227287,JB147188,10/08/2017 03:00:00 AM,092XX S RACINE AVE,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,RESIDENCE,false,false,...,21,73,02,None,None,2017,02/11/2018 03:57:41 PM,None,None,None
2,11227583,JB147595,03/28/2017 02:00:00 PM,026XX W 79TH ST,0620,BURGLARY,UNLAWFUL ENTRY,OTHER,false,false,...,18,70,05,None,None,2017,02/11/2018 03:57:41 PM,None,None,None
3,11227293,JB147230,09/09/2017 08:17:00 PM,060XX S EBERHART AVE,0810,THEFT,OVER $500,RESIDENCE,false,false,...,20,42,06,None,None,2017,02/11/2018 03:57:41 PM,None,None,None
4,11227634,JB147599,08/26/2017 10:00:00 AM,001XX W RANDOLPH ST,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,HOTEL/MOTEL,false,false,...,42,32,02,None,None,2017,02/11/2018 03:57:41 PM,None,None,None


### Count the total number of instance before preprocessing:

In [23]:
dataset.count()

7092612

### Drop the unused columns:


In [24]:
dataset = dataset.drop('ID')\
            .drop('Case Number')\
            .drop('FBI Code')\
            .drop('Updated On')\
            .drop('IUCR')\
            .drop('X Coordinate')\
            .drop('Y Coordinate')\
            .drop('Location')

In [25]:
# Dropping some NA values for now
dataset = dataset.na.drop()
dataset.count()

6413586

### Add the "DateSimple" column to dataframe:

In [26]:
dataset=dataset\
            .withColumn("DateSimple", F.to_date(F.split(dataset.Date, " ")[0], "MM/dd/yyyy"))
dataset.limit(10).toPandas().head()

,Date,Block,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,Year,Latitude,Longitude,DateSimple
0,04/10/2019 04:37:00 PM,102XX S VERNON AVE,SEX OFFENSE,AGG CRIMINAL SEXUAL ABUSE,"SCHOOL, PUBLIC, BUILDING",false,false,0511,005,9,49,2019,41.708589,-87.612583094,2019-04-10
1,04/12/2019 04:08:00 PM,032XX N KEELER AVE,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,false,true,1731,017,30,16,2019,41.940297617,-87.732066473,2019-04-12
2,04/19/2019 01:57:00 PM,002XX N LARAMIE AVE,OFFENSE INVOLVING CHILDREN,AGGRAVATED CRIMINAL SEXUAL ABUSE BY FAMILY MEMBER,RESIDENCE,false,true,1532,015,28,25,2019,41.884865037,-87.755230327,2019-04-19
3,04/25/2019 05:20:00 PM,108XX S DR MARTIN LUTHER KING JR DR,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,false,true,0513,005,9,49,2019,41.697609261,-87.613507612,2019-04-25
4,05/13/2019 05:26:00 PM,090XX S RACINE AVE,ASSAULT,SIMPLE,STREET,false,false,2222,022,21,73,2019,41.729973132,-87.653166753,2019-05-13


### Add the "Week" column to dataframe:

In [27]:
dataset=dataset.withColumn("Week", F.dayofweek(dataset.DateSimple))
dataset.limit(10).toPandas().head()

,Date,Block,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,Year,Latitude,Longitude,DateSimple,Week
0,04/10/2019 04:37:00 PM,102XX S VERNON AVE,SEX OFFENSE,AGG CRIMINAL SEXUAL ABUSE,"SCHOOL, PUBLIC, BUILDING",false,false,0511,005,9,49,2019,41.708589,-87.612583094,2019-04-10,4
1,04/12/2019 04:08:00 PM,032XX N KEELER AVE,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,false,true,1731,017,30,16,2019,41.940297617,-87.732066473,2019-04-12,6
2,04/19/2019 01:57:00 PM,002XX N LARAMIE AVE,OFFENSE INVOLVING CHILDREN,AGGRAVATED CRIMINAL SEXUAL ABUSE BY FAMILY MEMBER,RESIDENCE,false,true,1532,015,28,25,2019,41.884865037,-87.755230327,2019-04-19,6
3,04/25/2019 05:20:00 PM,108XX S DR MARTIN LUTHER KING JR DR,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,false,true,0513,005,9,49,2019,41.697609261,-87.613507612,2019-04-25,5
4,05/13/2019 05:26:00 PM,090XX S RACINE AVE,ASSAULT,SIMPLE,STREET,false,false,2222,022,21,73,2019,41.729973132,-87.653166753,2019-05-13,2


### Add the "Month" column to dataframe

In [28]:
dataset=dataset.withColumn("Month", F.month(dataset.DateSimple))
dataset.limit(10).toPandas().head()

,Date,Block,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,Year,Latitude,Longitude,DateSimple,Week,Month
0,04/10/2019 04:37:00 PM,102XX S VERNON AVE,SEX OFFENSE,AGG CRIMINAL SEXUAL ABUSE,"SCHOOL, PUBLIC, BUILDING",false,false,0511,005,9,49,2019,41.708589,-87.612583094,2019-04-10,4,4
1,04/12/2019 04:08:00 PM,032XX N KEELER AVE,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,false,true,1731,017,30,16,2019,41.940297617,-87.732066473,2019-04-12,6,4
2,04/19/2019 01:57:00 PM,002XX N LARAMIE AVE,OFFENSE INVOLVING CHILDREN,AGGRAVATED CRIMINAL SEXUAL ABUSE BY FAMILY MEMBER,RESIDENCE,false,true,1532,015,28,25,2019,41.884865037,-87.755230327,2019-04-19,6,4
3,04/25/2019 05:20:00 PM,108XX S DR MARTIN LUTHER KING JR DR,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,false,true,0513,005,9,49,2019,41.697609261,-87.613507612,2019-04-25,5,4
4,05/13/2019 05:26:00 PM,090XX S RACINE AVE,ASSAULT,SIMPLE,STREET,false,false,2222,022,21,73,2019,41.729973132,-87.653166753,2019-05-13,2,5


In [29]:
### Add the "Day" column to dataframe

In [30]:
#1- SUNDAY , 2 - MONDAY .... 7- SATURDAY
dataset=dataset.withColumn("Day", F.dayofweek(dataset.DateSimple))
dataset.limit(10).toPandas().head()

,Date,Block,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,Year,Latitude,Longitude,DateSimple,Week,Month,Day
0,04/10/2019 04:37:00 PM,102XX S VERNON AVE,SEX OFFENSE,AGG CRIMINAL SEXUAL ABUSE,"SCHOOL, PUBLIC, BUILDING",false,false,0511,005,9,49,2019,41.708589,-87.612583094,2019-04-10,4,4,4
1,04/12/2019 04:08:00 PM,032XX N KEELER AVE,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,false,true,1731,017,30,16,2019,41.940297617,-87.732066473,2019-04-12,6,4,6
2,04/19/2019 01:57:00 PM,002XX N LARAMIE AVE,OFFENSE INVOLVING CHILDREN,AGGRAVATED CRIMINAL SEXUAL ABUSE BY FAMILY MEMBER,RESIDENCE,false,true,1532,015,28,25,2019,41.884865037,-87.755230327,2019-04-19,6,4,6
3,04/25/2019 05:20:00 PM,108XX S DR MARTIN LUTHER KING JR DR,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,false,true,0513,005,9,49,2019,41.697609261,-87.613507612,2019-04-25,5,4,5
4,05/13/2019 05:26:00 PM,090XX S RACINE AVE,ASSAULT,SIMPLE,STREET,false,false,2222,022,21,73,2019,41.729973132,-87.653166753,2019-05-13,2,5,2
